## AWS GLUE REVIEW

*Alvaro Santoma, Gustau Solé. Basetis 2018*

En Agosto de 2017 AWS sacó el servicio AWS Glue. Ellos mismos lo definen como un servicio de ETL completamente gestionado.

https://docs.aws.amazon.com/glue/latest/dg/what-is-glue.html

Vamos a explicar como funciona y que es capaz de hacer. Partimos de unos datos de origen como archivos crudos. Podemos usar un crawler para estructurarlos en una base de datos. Si nuestro punto de partida ya es una base de datos estructurada nos ahorramos el crawler. Sobre la base de datos se pueden realizar ciertas transformaciones que funcionan sobre la tecnología Spark, que corre sobre Hadoop. Por lo tanto aprovecha el sistema de archivos distribuidos. Es decir, está pensado para cantidades grandes de datos que estén en clusters o el cloud. La salida de resultados puede ser otro archivo crudo o una base de datos. Origen y destino pueden estar en cualquier sitio, pero es más cómodo si todo está en el entorno AWS.

Veamos todo paso a paso:

**1. CRAWLER:**

Lo primero que necesitamos es alimentar un catálogo (Glue Data Catalog) con los datos de origen. En el catálogo no están los datos, solo los metadatos de indexación, esquema y estadísticas de los datos de origen. Si partimos de un archivo plano, lo más cómodo (y de pago) es usar un Crawler que nos va a parsear el archivo y alimentar el catálogo. Es decir, va a detectar la estructura del archivo (ya sea csv, parquet, txt u otros), y lo presentará como una database estructurada con filas (registros) y columnas (atributos).

Hay que especificar si el archivo origen está en tu propio repoositorio S3 o el S3 de otro usuario. Existe la opción de apuntar a otros repositorios o bases de datos mediante protocolo JDBC.

Es importante tener creado un Role con Full Access para poder realizar esta operación. Si no está bien configurado, el crawler aparentemente se ejecuta, pero al final no presenta ninguna tabla de resultados y no da ninguna alerta ni señal de error, con lo cual se puede perder mucho tiempo.

Si todo ha ido bien, el Crawler da como resultado una o más tablas que aparecen en "Databases" y "Tables".

Los crawlers son de pago. Se paga mientras dura su ejecución con un mínimo de 10 minutos. Se paga por DPU/hora a 0,44\$. Por lo tanto un trabajo mínimo con 10 DPU's cuesta 0,70\$. Se puede mantener los crawlers en espera sin costo. Se puede programar el crawler para que se ejecute cada cierto tiempo a modo de refresco de la base de datos cuando cambian los datos de origen.

**2. JOBS**

Jobs se refiere propiamente al trabajo de ETL. El origen debe ser una tabla de la database (en nuestro caso la tabla que nos ha creado el crawler). Debemos especificar la ubicación y tipo del archivo de salida (csv, avro, parquet, json,...). Los jobs nos permiten transformar la base de datos de origen según nos convenga. 

Hay tres niveles de transformaciones:

- Sin programar nada: Mediante una interfaz gráfica se nos muestran todos los campos de la base de datos de origen enlazados con los campos de la base de datos transformada. De forma sencilla se pueden descartar y ordenar, mapear a otros campos y cambiar el tipo de dato entre una serie de tipos preconfigurados. El programa generará el código necesario para realizar estas operaciones (en python o scala a escoger). EL código generado se puede modificar.

- Funciones de AWS Glue predefinidas. Una vez ya tenemos código frente a nosotros podemos usar unas funciones predefinidas que nos generan la estructura del código que requieren. Sin embargo para que funcionen tenemos que introducir a mano parámetros, condiciones, etc... además de colocarlos en el lugar adecuado del pipeline de transformación. Las funciones predefinidas añaden alguna funcionalidad a las funciones anteriores, como por ejemplo filtros (wheres)

- Funciones de SparkSQL. Por último, tenemos liberad total para realizar las transformaciones y agregados que queramos mediante el uso de funciones de SparkSQL. Para ello debemos convertir el DataFrame de Glue a DataFrame de Spark, hacer las transformaciones que se requieran y volver a convertir el DataFrame a Glue. Esto requiere introducir código (python o scala) y conocer el framework de SparkSQL. Se puede realizar cualquier transformación: agregados, agrupaciones, etc...

Los Jobs son de pago y valen igual que los crawlers (precio por DPU/hora). Solo se paga por el rato que se ejecuta el job con un mínimo de 10 minutos.


**3. EXPLOTACION DE LA SALIDA**

Como salida de resultados obtenemos archivos, en general particionados, que se almacenan en S3 u otro lugar que se especifique. Estos archivos pueden ser consumidos por cualquier aplicación. Si la salida es muy grande puede almacenarse en formato Parquet, pasarse de nuevo por un crawler para crear una nueva base de datos y realizar queries sobre esta con, por ejemplo, AWS Athena. Si ya hemos reducido la salida mediante agregados puede ser consumido por alguna aplicación de visualización. En el entorno AWS se ofrece QuickSight.

**4. RESUMEN**

La ventaja de AWS Glue se obtiene cuando los datos de origen y de destino están en AWS, ya que todo queda en el mismo entorno y el trabajo es sencillo. No obstante también puede operar con datos externos aunque no lo hemos probado.

Los crawlers convierten archivos crudos en un catálogo con esquema y apariencia de base de datos de esquema relacional.

Si no se quiere programar nada son pocas las transformaciones que se pueden hacer más allá del trabajo que hace el crawler, solamente refinarlo por si se ha equivocado en algún tipo de dato, poner nombres a los campos y descartar los que no interesen. **No es posible hacer transformaciones más elaboradas sin programar.**

Si los datos a tratar no son grandes no tiene sentido el uso de Glue ya que la gracia es aprovechar el procesamiento distribuido de Spark. Aun así el crawler puede ser útil para hacer un parseado del archivo crudo, pero debe haber otras herramientas en local para hacerlo.


**5. EJEMPLO: WINES DATASET**

Vamos a documentar de principio a fin un proceso de ETL con Amazon Glue. El trabajo consiste en procesar un archivo de texto que contiene 10.000 registros de tipos de vino con una serie de atributos que describen sus propiedades físicas y químicas y sacar algunos agregados. Es un toy-example que no requiere de procesamiento distribuido, pero AWS Glue escala de forma automática, así que el ejemplo sirve para ilustrar cualquier trabajo similar con otra volumetria mayor. Sin embargo los costes serían obviamente mayores.  

Para poder comparar, el trabajo está hecho con **PySpark** y se adjunta al final de la notebook como anexo. Nuestro objetivo es realizar el mismo trabajo con AWS Glue desde su interfaz drag and drop para evaluar sus posibilidades. 

Se ha subido el archivo "wines.10m.txt" a S3, en concreto a s3://glue-sample-target-alvaro/wines.10m.txt, que será nuestro Data Source.

- Se ha creado un **crawler** que ha leído los datos de nuestra Data Source. Automáticamente nos crea una tabla en la base de datos que le hayamos asignado. Hay que tener cuidado con los roles, mejor darle Full Access (ver apartado 1).<br>

- Una vez ya tenemos la tabla procedemos a hacer un **job**. En el establecemos la tabla de origen y de destino, junto con el formato de salida de esta (elegimos JSON). La primera pantalla nos permite eliminar aquellos campos que no nos interesan y modificar los nombres de los campos (elegimos las columnas 1 y 5 y les damos nombre "type" y "ash" respectivamente). Nos hemos dado cuenta que algunos de los campos del schema no los ha entendido bien el crawler, en vez de ser Doubles nos los asignaba como Strings. Es importante conocer nuestro dataset para hacer este tipo de chequeos, ya que el crawler, a la vista de los resultados no es 100% fiable. En cualquier caso esta interfaz nos permite forzar el tipo de dato manualmente.
    
- Superada la interfaz gráfica, AWS Glue nos ofrece un código generado automáticamente que recoge aquello que previamente hemos especificado mediante clicks. Pero para nuestro propósito (sumar el atributo "ash" agrupado por "type") es insuficiente. Respecto al script que se genera automáticamente hemos introducido los siguientes cambios.
    
    -- Se han importado una serie de librerías de pyspark sql y aws glue:
        from awsglue.transforms import * <br>
        from awsglue.utils import getResolvedOptions<br>
        from pyspark.context import SparkContext<br>
        from awsglue.context import GlueContext<br>
        from awsglue.job import Job<br>
        from awsglue.dynamicframe import DynamicFrame<br>
        from pyspark import SparkConf, SparkContext<br>
        from pyspark.sql import * <br>
        from pyspark.sql.types import * <br>
        from pyspark.sql.functions import * <br>
        from pyspark.sql.dataframe import * <br>
        from pyspark.sql.readwriter import * <br>
        from pyspark.sql.utils import * <br>

    -- Entre el último DynamicDataFrame generado automáticamente y la salida de resultados hemos incorporado una transformación adicional que consiste en convertir el DynamicDataFrame de glue a un DataFrame de SparkSQL mediante la funcion toDF(). Mediante las funciones de la librería de SparkSQL hacemos un groupBy, agragación y ordenación. El resultado lo volvemos a convertir a un DynamicDataFrame de glue mediante DynamicDataFrame.fromDF(). Lo anterior es para usar las opciones de escritura de los resultados que ofrece glue.

        sparkdf =dropfields2.toDF().groupBy("type").sum().sort("type")<br>
        dynamicdataframe = DynamicFrame.fromDF(sparkdf, glueContext, 'dynamicdataframe')<br>
    

- Ver los **resultados**. Los resultados se almacenan en el Data target que hayamos definido en el Job. En nuestro caso nos ha generado 3 JSON con una fila cada 1, lo cual es absurdo en este caso, pero está pensado para distribuir la salida de resultados en el cluster.<br>

    Estos archivos de resultados se pueden recomponer en local o dejar en distribuido y hacer el análisis de datos con la herramienta que creamos necesaria, ya sea de visualización o de nuevas queries según sea el caso. 



A continuación se muestra todo el código del Job de Glue

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.dataframe import *
from pyspark.sql.utils import *

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)
## @type: DataSource
## @args: [database = "wine", table_name = "wines_10m_txt", transformation_ctx = "datasource0"]
## @return: datasource0
## @inputs: []
datasource0 = glueContext.create_dynamic_frame.from_catalog(database = "wine", table_name = "wines_10m_txt", transformation_ctx = "datasource0")
## @type: ApplyMapping
## @args: [mapping = [("col0", "string", "type", "string"), ("col4", "double", "ash", "double")], transformation_ctx = "applymapping1"]
## @return: applymapping1
## @inputs: [frame = datasource0]
applymapping1 = ApplyMapping.apply(frame = datasource0, mappings = [("col0", "string", "type", "string"), ("col4", "double", "ash", "double")], transformation_ctx = "applymapping1")

## @type: CustomSparkSQL
## @args: []
## @return: dynamicdataframe
## @inputs: [frame = applymapping1]
sparkdf =applymapping1.toDF().groupBy("type").sum().sort("type")
dynamicdataframe = DynamicFrame.fromDF(sparkdf, glueContext, 'dynamicdataframe')

## @type: DataSink
## @args: [connection_type = "s3", connection_options = {"path": "s3://glue-sample-target-alvaro"}, format = "json", transformation_ctx = "datasink2"]
## @return: datasink2
## @inputs: [frame = dynamicdataframe]
datasink2 = glueContext.write_dynamic_frame.from_options(frame = dynamicdataframe, connection_type = "s3", connection_options = {"path": "s3://glue-sample-target-alvaro"}, format = "json", transformation_ctx = "datasink2")
job.commit()

### ANEXO: EL MISMO EJEMPLO HECHO CON PYSPARK

En esta notebook se ilustra el funcionamiento del módulo SQL de SPARK en Python. El ejercicio consiste en procesar un archivo csv que contiene 10.000 registros, cada uno de los cuales corresponde a vino, descrito mediante 14 variables (type, region, y 12 medidas de propiedades físico-químicas: alc, m_acid, ash, alc_ash, mgn, t_phenols, flav, nonflav_phenols, proant, col, hue, od280/od315 y proline). Transformaremos el archivo crudo en un dataset estructurado y realizaremos un par de queries de ejemplo en el. Es un ejemplo de juguete que ejecutaremos en local, pero al estar usando Spark, podríamos escalar el problema todo lo que quisiéramos si tuviéramos el archivo de origen distribuido en un cluster.

In [6]:
#imports
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

El punto de partida es crear un SparkContext y una SparkSession.

In [7]:
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

Leemos el archivo de origen obteniendo un RDD, que vamos transformando en pipeline con los siguientes pasos:
1. Lectura del archivo (obtenemos un RDD de Strings, cada uno es una linea del CSV)
2. Split por coma (obtenemos un RDD de arrays de Strings con los distintos campos de cada linea)
3. Casteamos cada campo para que sea del tipo adecuado, strings, integer y float. 

In [8]:
winesRDD = sc.textFile("resources/wines.10m.txt")\
    .map(lambda l: l.split(","))\
    .map(lambda l: [l[0], int(l[1]), float(l[2]), float(l[3]), float(l[4])\
                    , float(l[5]), float(l[6]), float(l[7]), float(l[8]), float(l[9])\
                    , float(l[10]), float(l[11]), float(l[12]), float(l[13]), float(l[14])])
    
print(winesRDD.first())

['type_1', 8, 10.641847, 3.970313, 2.064254, 17.814904, 64.504434, 4.906606, 0.239978, 3.301748, 0.136569, 2.998473, 1.540317, 1.691263, 1289.861234]


El RDD es una estructura de datos superflexible que nos permite hacer todo tipo de transformaciones y agregados. Podemos trabajarlo en la versión simple de un elemento por registro (como el winesRDD) o la versión Con el con pares key-value que permite hacer agrupaciones, joins, etc... Todo ello responde al esquema lógico heredado de MapReduce. Las funciones aplicables a un RDD en python se pueden encontrar aquí:

http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD

Si nuestro RDD tiene una estructura definida nos puede ser más cómodo transformarlo a un DataFrame, estructura asociada a la librería SparkSQL. Nos permite hacer transformaciones muy similares a las que se hacen con RDD's pero con una lógica SQL, que le es más cercana a muchos usuarios.

Un DataFrame no es más que una tabla con una estructura definida, en la que todas las filas tienen la misma estructura, las columnas tienen un nombre y todos los elementos de cada columna son del mismo tipo. Es decir, es una tabla típica con las que se trabaja en SQL.

Para convertir un RDD a un DataFrame hay que explicitar el esquema. Vamos a construirlo:

In [9]:
fields = [StructField("type", StringType(), True),\
          StructField("region", IntegerType(), True),\
          StructField("alc", FloatType(), True),\
          StructField("m_acid", FloatType(), True),\
          StructField("ash", FloatType(), True),\
          StructField("alc_ash", FloatType(), True),\
          StructField("mgn", FloatType(), True),\
          StructField("t_phenols", FloatType(), True),\
          StructField("flav", FloatType(), True),\
          StructField("nonflav_phenols", FloatType(), True),\
          StructField("proant", FloatType(), True),\
          StructField("col", FloatType(), True),\
          StructField("hue", FloatType(), True),\
          StructField("od280/od315", FloatType(), True),\
          StructField("proline", FloatType(), True)]
schema = StructType(fields)

Una vez construido el esquema ya podemos construir el DataFrame:

In [10]:
winesDF = spark.createDataFrame(winesRDD,schema)
winesDF.show(5)

+------+------+---------+--------+--------+---------+--------+---------+--------+---------------+--------+--------+--------+-----------+---------+
|  type|region|      alc|  m_acid|     ash|  alc_ash|     mgn|t_phenols|    flav|nonflav_phenols|  proant|     col|     hue|od280/od315|  proline|
+------+------+---------+--------+--------+---------+--------+---------+--------+---------------+--------+--------+--------+-----------+---------+
|type_1|     8|10.641847|3.970313|2.064254|17.814903|64.50443| 4.906606|0.239978|       3.301748|0.136569|2.998473|1.540317|   1.691263|1289.8612|
|type_3|   262|12.368596|3.012189|2.925376|27.720331|96.28931| 2.179099|0.482674|       1.426959|0.665298|1.420153|1.037437|   1.051302|1754.7133|
|type_2|     0|10.835006|1.611363|2.683955| 17.18269|89.97082| 0.138075|0.202834|       1.389415| 0.83666|4.213477|0.420737|   2.791477|1930.2006|
|type_1|     8|  12.6945|2.997357|2.991452|20.622334|84.50368| 1.894055|0.422536|       4.235345|0.313358|0.698229|0.0

Con el DataFrame podemos hacer la mayoría de las operaciones que se pueden hacer con SQL. Y se pueden hacer de dos maneras. 

1. Usando las funciones propias de los DataFrames que se pueden encontrar aquí:

http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame

2. Creando una View y lanzando sentencias SQL sobre ella

Veamos dos queries mediante ambos métodos:

##### QUERY 1: Obtener la suma de ash, ordenado por cada tipo de vino.

In [11]:
#Method 1: DataFrame Functions

result = winesDF\
    .select('type', winesDF.ash)\
    .groupBy('type')\
    .sum()\
    .sort('type')

result.show()

+------+-----------------+
|  type|         sum(ash)|
+------+-----------------+
|type_1|68981.18038606644|
|type_2|68327.76287817955|
|type_3|68298.86391997337|
+------+-----------------+



In [12]:
#Method 2: SQL command

winesDF.createOrReplaceTempView("wines")

sqlDF = spark.sql("SELECT type, SUM(ash) AS SUM_ash FROM wines GROUP BY type ORDER BY type")
sqlDF.show()

+------+-----------------+
|  type|          SUM_ash|
+------+-----------------+
|type_1|68981.18038606644|
|type_2|68327.76287817955|
|type_3|68298.86391997337|
+------+-----------------+



##### QUERY 2: Obtener la media de alcohol de los vinos con acidez mayor a 2, ordenado por cada tipo de vino.

In [13]:
#Meyhod 1: DataFrame Functions

result = winesDF\
    .where(winesDF.m_acid > 2.0)\
    .select('type', winesDF.alc)\
    .groupBy('type')\
    .avg()\
    .sort('type')

result.show()

+------+------------------+
|  type|          avg(alc)|
+------+------------------+
|type_1|12.507112678371088|
|type_2| 12.50832716678349|
|type_3|12.499955222377686|
+------+------------------+



In [14]:
#Method 2: SQL command

winesDF.createOrReplaceTempView("wines")

sqlDF = spark.sql("SELECT type, AVG(alc) AS alc FROM wines WHERE m_acid > 2.0 GROUP BY type ORDER BY type")
sqlDF.show()

+------+------------------+
|  type|               alc|
+------+------------------+
|type_1|12.507112678371088|
|type_2| 12.50832716678349|
|type_3|12.499955222377686|
+------+------------------+

